#Basics of file system in databricks 


In [0]:
# dbutils.help()
# dbutils.fs.help()
# dbutils.fs.help("cp")

# dbutils.fs.ls("/")  #command to list all directory, FileStore, mnt are only default along with dataset and results folders.
# display(dbutils.fs.ls("/")) # create a new folder in dbfs(databricks file system)
# dbutils.fs.mkdirs("dbfs:/temp")
# Remember backslash before first folder.
# dbutils.fs.rm("dbfs:/temp/", True)  #command to delete any folders # The second argument, True, indicates recursive deletion
# dbutils.fs.rm("dbfs:/")


In [0]:
#HOW TO DELETE ALL THE FILES INSIDE FOLDER RECURSIVELY and FS file system 

# %fs ls /
# stick with dbt package (dbutils)
# dbutils.fs.rm("dbfs:/mnt/readeachjson1/*", True)
# dbutils.fs.ls("dbfs:/mnt/readeachjson1")
#dbutils.fs.rm("/mnt/readeachjson1/*", recurse=True)
#dbutils.fs.rm("dbfs:/mnt/readeachjson1/*", recurse=True)


#Create DataFrame using Pandas and Pyspark

In [0]:
import pandas as pd

data = {
    "id": [1, 2, 3, 4, 5],
    "name": ['Alice', 'Bob', 'Char', 'Diana', 'Emily'],
    "age": [10, 20, 30, 40, 50],
    "department": ['HR', 'IT', 'Finance', 'HR', 'IT']
}

pdf = pd.DataFrame(data)
display(pdf)
#How to cwrite pandas dataframe into dbfs 
# Write the DataFrame to a CSV file in DBFS 
#df.to_csv('dbfs:/FileStore/a.csv', index=False)



id,name,age,department
1,Alice,10,HR
2,Bob,20,IT
3,Char,30,Finance
4,Diana,40,HR
5,Emily,50,IT


Pyspark Dataframe 

In [0]:
from pyspark.sql import SparkSession
from pyspark import Row

spark =SparkSession.builder.appName("Pysaprk practice").getOrCreate()
# display(spark) 

data=[
    Row(id=1,F_name="Praful", L_name='BS',Salary=50000),
    Row(id=2,F_name='Rakesh', L_name='jkl',Salary=60000),
    Row(id=3,F_name='Suvarna', L_name='ling',Salary=70000)
]
# df=spark.createDataFrame(data)
# df.write.csv("/temp/employee.csv", header=True,mode="overwrite")
# display(df)

Look at the difference while creating df, in pandas you take data as data ={'c':[v1,v2..]}
%md In spark data as list of rows, column name don't come inside double quotes data =[ Row(c=V, c='v2', c3='v3',c4=v4)]

#Create a DF using Data, Data with schema, csv, txt, json, parquet etc.

In [0]:
# with schema 
from pyspark.sql import SparkSession 
from pyspark.sql import Row
spark=SparkSession.builder.appName("Pyspark").getOrCreate()
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
data=[
    Row(id=1,F_name="Praful", L_name='BS',Salary=50000),
    Row(id=2,F_name='Rakesh', L_name='jkl',Salary=60000),
    Row(id=3,F_name='Suvarna', L_name='ling',Salary=70000)
]

schema =  StructType([StructField("id", IntegerType(), True),
                      StructField("F_name", StringType(), True),
                      StructField("L_name", StringType(), True),
                      StructField("Salary", IntegerType(), True) ])


# df=spark.createDataFrame(data, schema)
# display(df)

#create df using 
sdf = spark.read.csv("/temp/s_employee_data.csv", header=True)
display(sdf)
# sdf.show()
sdf.printSchema()



id,name,age,department
3,Charlie,35,Finance
1,Alice,30,HR
4,David,28,HR
2,Bob,25,IT
5,Eva,32,IT


root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- department: string (nullable = true)



How to access rows and columns of pysaprk dataframe and how diffrent from pandas 

In [0]:

import pandas as pd 
from pyspark.sql.functions import col

#pandas ways of manupulating data/df

# pdf # simply running the DataFrame variable in a cell will display the DataFrame's contents directly 
# display(sdf) # you typically use the display() function to visualize the contents of a DataFrame.

# In pandas df, you use direct indexing of df's, df['column_name'] or df.loc[row_label].
# In use functions like select and filter to specify what data you want from the DataFrame.

# working with df , don't assign the filter condition to any variables, it might mess it up.
# pandas way of accessing columns, rows , boolean kind(vector)

# pdf.loc[0,'age'] or df.at[0,'A']  #filter the pdf for single value(row, column)
# pdf['age'] or pdf.age #filter df based on single column name.
# pdf.loc[0] #filter df based on row number.

# pdf.loc[:,'id':'age'] # filter df based on column range.
# pdf.loc[0:2, :] #filter df based on row range 

# pdf.loc[pdf.age >30] #boolean operation #here I shouldn't assign to an variable
# pdf.loc[pdf.age >30, 'department']='Mang' #boolean operation # chnages whole df based on condition.  


#spark way of manipulating data/df
 
# display(sdf.take(5)) or sdf.head(5)
# sdf.columns

# sdf.select("age").show()  #Filter one column
# sdf['age'] # this is error because column in not callable  # this is just an column object.

# sdf.filter(sdf.age==35).show()  # Filter one row
# sdf.filter(sdf.age==35).select('department').show() #use .first()[0] insted of show [You just get one value here not df] # filter one value(row, column)

# sdf.select("id","age").show() #filter multiple columns
# sdf.filter(sdf.age>30).show() #filter range of rows

# Now focus on some spark functions # col, 
# select cols using function 
# import pyspark.sql.functions as F
# sdf.select(F.col("id"), F.col("age")).show()

 
# using bitwise(&) and pipe(|) operators 

# sdf.filter((sdf['age']>30) & (sdf['id'] >1)).show()
# sdf.filter((sdf.select('age')>30)).show() # this also error 

# sdf.filter(sdf['age']>30).count() #count of records 

# from pyspark.sql.functions import when
# sdf.withColumn("Department", when(sdf.age>30, "Fire")).show() # this overite the valkue in column already existing column department

# from pyspark.sql.functions import when
# sdf.withColumn("Dep", when(sdf.age>30, "Fire")).show() # this add new column to sdf (best way to go)

# you can use groupby, agg function(sum, mean, avg etc), distinct ...
# sdf.groupBy('department').count().show()  #grouping the data based in column  

# df=df.dropDuplicates()  # to remove duplicates 

# df.write.csv("./data/my_dataset.csv") #writing data into file syatem 


+----------+-----+
|department|count|
+----------+-----+
|   Finance|    1|
|        HR|    2|
|        IT|    2|
+----------+-----+



In [0]:
d# Using col method
column_data = sdf.col("column_name")


